# SPARK hackfest-in-a-box for TELCO - LAB 2

The objetive is this lab is to perform model training with Spark MLLib Random Forest Classification model for the customer churn. Random forests are ensembles of decision trees. Random forests are one of the most successful machine learning models for classification and regression. They combine many decision trees in order to reduce the risk of overfitting. Like decision trees, random forests handle categorical features, extend to the multiclass classification setting, do not require feature scaling, and are able to capture non-linearities and feature interactions.

`spark.mllib` supports random forests for binary and multiclass classification and for regression, using both continuous and categorical features. `spark.mllib` implements random forests using the existing decision tree implementation. Please see the decision tree guide for more information on `trees.prediction`

## 1 Initial data loading

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F
import pandas as pd
import sys, logging, argparse, random, tempfile, json
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StructType, DoubleType, StringType
from pyspark.sql.functions import lit

In [2]:
print('....Setting input variables')
projectNbr = "YOUR_PROJECT_NBR"
projectID = "YOUR_PROJECT_ID"
appBaseName = "customer-churn-model"
appNameSuffix = "training"
appName = f"{appBaseName}-{appNameSuffix}"
modelBaseNm = appBaseName
bqDatasetNm = f"{projectID}.customer_churn_ds"
operation = appNameSuffix
bigQuerySourceTableFQN = f"{bqDatasetNm}.training_data_step_3"

....Setting input variables


In [3]:
print('....Initializing spark & spark configs')
spark = SparkSession.builder.appName(appName).getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

....Initializing spark & spark configs


In [4]:
print('....Read the training dataset into a dataframe')
inputDF = spark.read \
    .format('bigquery') \
    .load(bigQuerySourceTableFQN)
# Typecast some columns to the right datatype
inputDF = inputDF.withColumn("monthly_charges", inputDF.monthly_charges.cast('float')) \
    .withColumn("total_charges", inputDF.total_charges.cast('float'))

....Read the training dataset into a dataframe


#### Q - Split the data in training and test datasets
* Use `randomSplit` to split the data into the `trainDF`, `testDF` dataframes

In [5]:
SPLIT_SEED = 6
SPLIT_SPECS = [0.8, 0.2]
print('....Splitting the dataset')
trainDF, testDF = inputDF._______INSERT_CODE_HERE_______

....Splitting the dataset


## 2 Feature engineering

Feature engineering or feature extraction or feature discovery is the process of using domain knowledge to extract features (characteristics, properties, attributes) from raw data. The motivation is to use these extra features to improve the quality of results from a machine learning process, compared with supplying only the raw data to the machine learning process.

#### Q - One hot encode the categorical columns
* One hot encode  each text categorical column that you have identified in the previous lab using first `stringIndexer` and then `OneHotEncoderEstimator`
* Do not fit the transformations yet, instead maintain a list with all the required steps

In [6]:
print('....Data pre-procesing: One hot encoding of categotical columns')
CATEGORICAL_COLUMN_LIST = ['gender', 'senior_citizen', 'partner', 'dependents', 'phone_service', 'multiple_lines',
                        'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support',
                        'streaming_tv', 'streaming_movies', 'contract', 'paperless_billing', 'payment_method']  
dataPreprocessingStagesList = []
for eachCategoricalColumn in CATEGORICAL_COLUMN_LIST:
    stringIndexer = StringIndexer(_______INSERT_CODE_HERE_______)
    # https://spark.apache.org/docs/3.0.0-preview/ml-migration-guide.html 
    if (spark.version).startswith("2."):
        from pyspark.ml.feature import OneHotEncoderEstimator
        encoder = OneHotEncoderEstimator(_______INSERT_CODE_HERE_______)
    elif (spark.version).startswith("3."):
        from pyspark.ml.feature import OneHotEncoder
        encoder = OneHotEncoder(_______INSERT_CODE_HERE_______)
    else:
        from pyspark.ml.feature import OneHotEncoder
        encoder = OneHotEncoder(_______INSERT_CODE_HERE_______)
    dataPreprocessingStagesList += _______INSERT_CODE_HERE_______

....Data pre-procesing: One hot encoding of categotical columns


#### Q - Generate labels for the output variable (churn)
* Use `stringIndexer` and generate labels for the churn column
* Do not fit the transformations yet, instead append the transformation to the previous list

In [7]:
print('....Data pre-procesing: Labels for target columns')
labelStringIndexer = StringIndexer(_______INSERT_CODE_HERE_______)
dataPreprocessingStagesList += _______INSERT_CODE_HERE_______

....Data pre-procesing: Labels for target columns


#### Q - Assemble all the features into the `features` columns
* Use `VectorAssembler` and a `features` column with the one hot encoded features + previous numerical features
* Do not fit the transformations yet, instead append the transformation to the previous list

In [8]:
NUMERIC_COLUMN_LIST = ['monthly_charges', 'total_charges']
print('....Generating features column')
assemblerInputs = NUMERIC_COLUMN_LIST + [c + "classVec" for c in CATEGORICAL_COLUMN_LIST]
featuresVectorAssembler = VectorAssembler(_______INSERT_CODE_HERE_______)
dataPreprocessingStagesList += _______INSERT_CODE_HERE_______

....Generating features column


## 3 Model training and testing

#### Q - Generate a Random Forest Classifier training step and the training pipeline
* Use `RandomForestClassifier` to define the training step
* Do not fit the transformations yet, instead append the transformation to a new list
* Generate a training pipeline using `Pipeline` putting together the feature engineering steps and the training step

In [9]:
print('....Model training')
modelTrainingStageList = []
rfClassifier = RandomForestClassifier(_______INSERT_CODE_HERE_______)
modelTrainingStageList += [rfClassifier]
print('....Generating the pipeline')
pipeline = Pipeline(_______INSERT_CODE_HERE_______) 

....Model training
....Generating the pipeline


#### Q - Fit the training pipeline
* Use `fit` to execute the training pipeline over the training dataset

In [10]:
print('....Fit the model')
pipelineModel = pipeline._______INSERT_CODE_HERE_______

....Fit the model


#### Q - Test the model with the test dataset
* Use `transform` to execute predidctions using the fitted model over the `testDF`

In [11]:
print('....Test the model')
predictionsDF = pipelineModel._______INSERT_CODE_HERE_______
predictionsDF.show(2, vertical=True)

....Test the model


22/09/23 07:55:57 WARN org.apache.spark.util.Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


-RECORD 0-----------------------------------------
 customer_id               | 0013-EXCHZ           
 gender                    | Female               
 senior_citizen            | 1                    
 partner                   | Yes                  
 dependents                | No                   
 tenure                    | 3                    
 tenure_group              | Tenure_0-12          
 phone_service             | Yes                  
 multiple_lines            | No                   
 internet_service          | Fiber optic          
 online_security           | No                   
 online_backup             | No                   
 device_protection         | No                   
 tech_support              | Yes                  
 streaming_tv              | Yes                  
 streaming_movies          | No                   
 contract                  | Month-to-month       
 paperless_billing         | Yes                  
 payment_method            | Ma

## 4 Model evaluation

#### Q - Calculate the area under ROC to asses the quality of the classifier
* Use `BinaryClassificationEvaluator` to calculathe the `areaUnderROC` metric
* What do you think about the value?

In [12]:
print('....Calculating area under the ROC curve')
evaluator = _______INSERT_CODE_HERE_______
evaluator.setRawPredictionCol("prediction")
evaluator.setLabelCol("label")
value = evaluator.evaluate(_______INSERT_CODE_HERE_______)

....Calculating area under the ROC curve


In [13]:
metricsDF = spark.createDataFrame( [("areaUnderROC",value)], ["metric_nm", "metric_value"]) 

In [15]:
metricsDF.show()

+------------+------------------+
|   metric_nm|      metric_value|
+------------+------------------+
|areaUnderROC|0.6376611418047882|
+------------+------------------+



**End of LAB 2**